In [1]:
import numpy as np
import random

import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

np.random.seed(42)
random.seed(42)

In [2]:
from DataExploration import DataExploration
from DecisionTree import DecisionTree
from NeuralNetwork import DNN
from Boosting import Boosting
from KNN import KNN
from NeuralNetworkKeras import NN_Keras
from SVM import SVM


C:\Users\matan\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


C:\Users\matan\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
Using TensorFlow backend.


In [3]:
#need to shuffle to respect iid principal for when we split the data
de = DataExploration()

twitter_df = de.prepare_twitter_df()
speed_dating_df = de.prepare_speed_dating_df()

(22223, 10)


(19276, 10)
(41499, 10)


C:\Users\matan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2903: DtypeWarning: Columns (3,10,11,15,16,17,18,19,39,40,41,42,43,44,51,52,53,54,55,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,107,109) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [4]:
X_train, X_test, y_train, y_test, X_train_nn, X_validation_nn, y_train_nn, y_validation_nn = de.get_train_test_validation(
    twitter_df, 'twitter')

In [5]:
best_params = {'dt__alpha': -0.01, 'dt__max_depth': 9, 'dt__min_samples_leaf': 7}
dt_clf = DecisionTree(model_name='dt_wip1', dataset_name='spam_twitter', X_test=X_test, y_test=y_test, best_params=best_params)
dt_clf.train(X_train, y_train)
y_pred = dt_clf.predict(X_test)
dt_clf.evaluate_model(y_test=y_test, y_pred=y_pred)

#dt_clfr.get_model().best_estimator_.named_steps['dt'].tree_.max_depth


[[3504  378]
 [ 316 4102]]
             precision    recall  f1-score   support

         -1       0.92      0.90      0.91      3882
          1       0.92      0.93      0.92      4418

avg / total       0.92      0.92      0.92      8300



In [6]:
dt_best_params = dt_clf.get_best_params()
print(dt_best_params)

{'dt__alpha': -0.01, 'dt__max_depth': 9, 'dt__min_samples_leaf': 7}


In [7]:
# dnn = DNN(n_classes=2, model_name='dnn_wip_3', dataset_name='spam_twitter')
# dnn.train(
#     learning_rate=0.05,
#     steps=1000,
#     batch_size=30,
#     hidden_units=[100, 100],
#     training_examples=X_train_nn,
#     training_targets=y_train_nn,
#     validation_examples=X_validation_nn,
#     validation_targets=y_validation_nn)
# 
# y_pred = dnn.predict(X_test, y_test)

In [8]:
best_params = {'boost__base_estimator__max_depth': 12, 'boost__n_estimators': 80}
# todo this is lower than pre pruning, figure out why
boosting_clf = Boosting(model_name='boosting_wip0', dataset_name='spam_twitter', X_test=X_test, y_test=y_test,
                        best_params=best_params)
boosting_clf.train(X_train, y_train)
y_pred = boosting_clf.predict(X_test)
boosting_clf.evaluate_model(y_test=y_test, y_pred=y_pred)


[[3419  463]
 [ 522 3896]]
             precision    recall  f1-score   support

         -1       0.87      0.88      0.87      3882
          1       0.89      0.88      0.89      4418

avg / total       0.88      0.88      0.88      8300



In [9]:
boosting_best_params = boosting_clf.get_best_params()
print(boosting_best_params)

{'boost__base_estimator__max_depth': 12, 'boost__n_estimators': 80}


In [10]:
best_params = {'knn__metric': 'manhattan', 'knn__n_neighbors': 10, 'knn__weights': 'distance'}
knn_clf = KNN(model_name='knn_wip0', dataset_name='spam_twitter', best_params=best_params)
knn_clf.train(X_train, y_train)
y_pred = knn_clf.predict(X_test)
knn_clf.evaluate_model(y_test=y_test, y_pred=y_pred)


[[3453  429]
 [ 531 3887]]
             precision    recall  f1-score   support

         -1       0.87      0.89      0.88      3882
          1       0.90      0.88      0.89      4418

avg / total       0.88      0.88      0.88      8300



In [11]:
knn_best_params = knn_clf.get_best_params()
print(knn_best_params)

{'knn__metric': 'manhattan', 'knn__n_neighbors': 10, 'knn__weights': 'distance'}


In [12]:
best_params = {'batch_size': 10, 'epochs': 1, 'layer_sizes': [12, 1], 'learning_rate': 0.0005,
               'loss': 'mean_squared_error'}

keras_nn_clf = NN_Keras(model_name='keras_nn_wip1', dataset_name='spam_twitter', best_params=best_params)
keras_nn_clf.train(X_train, y_train)
y_pred = keras_nn_clf.predict(X_test)
keras_nn_clf.evaluate_model(y_test=y_test, y_pred=y_pred)


ValueError: ('Could not interpret loss function identifier:', 0.0005)

In [13]:
keras_nn_best_params = keras_nn_clf.get_best_params()
print(keras_nn_best_params)

In [14]:
# keras_nn_clf = NN_Keras(model_name='keras_nn_optimizer_wip0', dataset_name='spam_twitter')
# keras_nn_clf.train(X_train, y_train, optimizer_tuning=True)
# y_pred = keras_nn_clf.predict(X_test)
# keras_nn_clf.evaluate_model(y_test=y_test, y_pred=y_pred)


In [15]:
# from sklearn.svm import SVC
# 
# linear_svm_clf = SVC(kernel='linear') #optimize kernel, regularization, gamma
# linear_svm_clf.fit(X_train, y_train)
# y_pred = linear_svm_clf.predict(X_test)

In [16]:

svm_clf = SVM(model_name='svm_wip0', dataset_name='spam_twitter')
svm_clf.train(X_train, y_train)
y_pred = svm_clf.predict(X_test)
svm_clf.evaluate_model(y_test=y_test, y_pred=y_pred)


In [ ]:
svm_best_params = svm_clf.get_best_params()
print(svm_best_params)

NameError: name 'svm_clf' is not defined

In [ ]:
X_train, X_test, y_train, y_test, X_train_nn, X_validation_nn, y_train_nn, y_validation_nn = de.get_train_test_validation(
    speed_dating_df, 'speed_dating')



NameError: name 'de' is not defined